In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#bleu score
# from nltk.translate.bleu_score import sentence_bleu

In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

src_text = "오늘 날씨가 정말 좋네요."
tokenizer.src_lang = "ko_KR"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

2025-06-18 16:24:12.572925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750263852.851972      34 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750263852.928111      34 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

It's really good weather today.


In [2]:
from datasets import load_dataset

ds = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/41.5M [00:00<?, ?B/s]

dev.json:   0%|          | 0.00/461k [00:00<?, ?B/s]

test.json:   0%|          | 0.00/448k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/166215 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1958 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1982 [00:00<?, ? examples/s]

In [11]:
from tqdm import tqdm

In [7]:
# !pip install evaluate
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.1 MB/s eta 0:00:00


In [8]:
import evaluate

In [9]:
bleuScorer = evaluate.load("bleu")
chrfScorer = evaluate.load("chrf")

In [21]:
weights = (0.25, 0.25, 0.25, 0.25) 
total = len(ds['test']['korean'])
bleu_score = 0
chrf_score = 0
for i in tqdm(range(len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i]
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    # print(predictions)
    reference = ds['test']['english'][i]
    # print(reference)
    # print(ds['test']['english'][i])
    bleu_score += bleuScorer.compute(predictions = [predictions], references = [reference])['bleu']
    chrf_score += chrfScorer.compute(predictions = [predictions], references = [reference], word_order = 2)['score']
    # print(f"blue_score = {bleu_score}, chrf_score = {chrf_score}")
    # print(score)
    if(i%20 == 0):
        print(f"i = {i}, score = {bleu_score}, chrf_score = {chrf_score}")
    # break
print(f"final score bleu_score = {bleu_score}, avg bleu_score = {bleu_score/len(ds['test']['korean'])}")
print(f"final score chrf_score = {chrf_score}, avg chrf_score = {chrf_score/len(ds['test']['korean'])}")

  0%|          | 1/1982 [00:04<2:16:53,  4.15s/it]

i = 0, score = 0.0, chrf_score = 27.228804568417488


  1%|          | 21/1982 [01:46<3:06:09,  5.70s/it]

i = 20, score = 5.961851194944208, chrf_score = 1161.0123846122451


  2%|▏         | 41/1982 [03:42<2:48:50,  5.22s/it]

...

i = 1960, score = 471.52453059613316, chrf_score = 99507.65223189417


100%|█████████▉| 1981/1982 [3:48:47<00:04,  4.18s/it]

i = 1980, score = 480.5518214265521, chrf_score = 100760.43733115445


100%|██████████| 1982/1982 [3:48:49<00:00,  6.93s/it]

final score bleu_score = 480.5518214265521, avg bleu_score = 0.2424580330103694
final score chrf_score = 100860.43733115445, avg chrf_score = 50.8882125787863


In [7]:
score

2.657753018626141e-228

In [8]:
total

1982

In [10]:
Bleu_score = score/total

print(f"Bleu score avg = {Bleu_score}, score = {score}, total = {total}")
#output: Bleu score avg = 1.3409450144430582e-231, score = 2.657753018626141e-228, total = 1982

Bleu score avg = 1.3409450144430582e-231, score = 2.657753018626141e-228, total = 1982


In [13]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing instal

In [14]:
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer
# Example texts
reference = "This is a reference text example."
candidate = "This is a candidate text example."
# BERTScore calculation
scorer = BERTScorer(model_type='bert-base-uncased')
P, R, F1 = scorer.score([candidate], [reference])
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore Precision: 0.9258, Recall: 0.9258, F1: 0.9258


In [19]:
# weights = (0.25, 0.25, 0.25, 0.25) 
total = len(ds['test']['korean'])
F1 = 0
P = 0
R = 0
bertScorer = BERTScorer(model_type='bert-base-uncased')
for i in tqdm(range(len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i]
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    pre, recall, cur_F1 = bertScorer.score([predictions], [reference])
    P += pre
    R += recall
    F1 += cur_F1
    # print(score)
    if(i%20 == 0):
        print(f"i = {i}, P = {P}, R = {R}, F1 = {F1}")
    # break

  0%|          | 1/1982 [00:02<1:32:35,  2.80s/it]

i = 0, P = tensor([0.3583]), R = tensor([0.3835]), F1 = tensor([0.3705])


  1%|          | 21/1982 [01:09<1:58:23,  3.62s/it]

i = 20, P = tensor([7.5117]), R = tensor([8.4170]), F1 = tensor([7.9198])


  2%|▏         | 41/1982 [02:22<1:47:06,  3.31s/it]

...

 99%|█████████▉| 1961/1982 [2:28:49<00:47,  2.24s/it]

i = 1960, P = tensor([721.7806]), R = tensor([847.5781]), F1 = tensor([776.9936])


100%|█████████▉| 1981/1982 [2:29:55<00:02,  2.75s/it]

i = 1980, P = tensor([728.2736]), R = tensor([855.1479]), F1 = tensor([783.9730])


100%|██████████| 1982/1982 [2:29:56<00:00,  4.54s/it]


In [21]:
print(f"F1 = {F1}, len = {total}, avg F1 = {F1/total}")
#output: F1 = tensor([784.3807]), len = 1982, avg F1 = tensor([0.3958])

F1 = tensor([784.3807]), len = 1982, avg F1 = tensor([0.3958])
